<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/YOLOv5_interference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**YOLOv5 interference**

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [32]:
%cd /gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco

/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco


In [33]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
weight = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt"

img = "/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/data/bullous_ehm0118.jpg"

In [37]:
!python detect.py  --weights $weight  --source $img

detect: weights=['/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/weights/eye_nii_2202_onecaseoneimage2_doctorcompare_yolov5s_epoch200_batch16_89.8p/last.pt'], source=/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/data/bullous_ehm0118.jpg, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/gdrive/MyDrive/Deep_learning/CorneAI_nagoya/yolo5_forcresco/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5 🚀 2021-12-20 torch 1.13.0+cu116 CPU

Fusing layers... 
Model Summary: 213 layers, 7034398 parameters, 0 gradients
Traceback (most recent call last):
  File "detect.py", line 334, in <module>
    main(opt)
  File "detect.py", line 329, in main
    run(**vars(opt))
  File "/usr/local/lib/python3.8/dist-packag